saved https://docs.google.com/spreadsheets/d/1GZayIFIrY2jdoxRIpk9KDTBLiE71VVtb7YAd5ZSYGR0/edit#gid=0
to `input/Example-Soil_NMDC_SampleMetadata.xlsx`
on 2021-07-13

use openpyxl for reading xlsx spreadsheet

https://github.com/ontodev/cogs for google sheets? may assume single tab

---

Assumptions:
- lowercased column header of select.* means no data?
- **also check for non-NAs?**

In [1]:
import pandas as pd

In [2]:
example_file = "../input/Example-Soil_NMDC_SampleMetadata.xlsx" 

The "Metadata" sheet in the NMDC Sample Metadata template has four header rows and color coded blocks of columns. The green and red sections have fixed columns. Here, the second though fourth rows provide elaboration on the expected content. _Remember, Pandas interprets the very first row as a dataframe's headers by default._

The columns and semantics of the blue "Required where applicable" section are variable. Users are supposed to fill in data required by the environmental packages describing their samples? All four of the header rows may need to be parsed/interpreted in order to convert the data rows into NMDC JSON data objects and record corresponding units.

These tabs will also have to be parsed, but they are not populated in the example file yet:
- ProjectInformation
- AssociatedResearchers
- EnvironmentalMetadata

In [12]:
# "Sample Identification"

green_id_col = ["ID"]

additional_green_col_names = [
    "sample_name",
    "investigation_type",
    "package",
    "source_mat_ID",
]

# "Required for all samples"
red_col_names = [
    "growth_facil",
    "geo_loc_name",
    "lat_lon",
    "collection_date",
    "samp_mat_process",
    "store_cond",
    "samp_store_temp",
    "samp_size",
    "env_broad_scale",
    "env_local_scale",
    "env_medium",
    "gold_ecosystem",
    "microbiome_taxonomy",
]


In [4]:
example_frame_structure = pd.read_excel(example_file, sheet_name=None)
example_sheet_names = example_frame_structure.keys()

In [5]:
# example_sheet_names

In [6]:
example_metadata_frame = pd.read_excel(example_file, sheet_name="Metadata")

In [7]:
example_metadata_headers = example_metadata_frame.loc[
    0:2,
]
example_metadata_headers

,ID,sample_name,investigation_type,package,source_mat_ID,growth_facil,geo_loc_name,lat_lon,collection_date,samp_mat_process,...,treatment.3,samp_collect_device,size_fract,select,select.1,select.2,select.3,select.4,select.5,select.6
0,"Universally unique ID (ex: IGSN, UUID)",Sample name/Laboratory ID,Analysis/ Data Type,Sample Type,Source Material ID,Growth Facility,Geographical Location Name,latitude;longitude,Collection Date and Time,sample material processing,...,watering_regm,slide_hammer_core,sieving,select,select,select,select,select,select,select
1,Field REQUIRED for ALL sample submission. Opti...,Human readable ID. This is the ID that will be...,This field is constrained to contain only a se...,This field contains the type of sample type as...,A unique identifier assigned to an original ma...,Type of facility where the sample was collecte...,Detailed geographic location of sampling site,The geographical origin of the sample as defin...,"The time of sampling, either as an instance (s...",Any processing applied to the sample during or...,...,treatment- watering regimen/schedule,sample collection device-slide hammer corer,size fraction-sieve size,,,,,,,
2,{text},{text},{text};{text};{text},drop down selection,NaN,drop down selection,{text},{float};{float},{YYYY-MM-DD},{text},...,{float} {unit};{Rn/start_time/end_time/duration},{text},{value}{units},,,,,,,


In [8]:
example_metadata_headers_cols = pd.Series(example_metadata_headers.columns)
# example_metadata_headers_cols

In [9]:
select_flag = example_metadata_headers_cols.str.match(
    "^select(\\.\\d+)?$", case=True, flags=0, na=None
)
select_cols = example_metadata_headers_cols[select_flag]
# select_cols

In [10]:
keeper_cols = example_metadata_headers_cols[~select_flag]
# keeper_cols
# list(keeper_cols)

In [14]:
example_metadata_content = example_metadata_frame.loc[
    3:,
]
example_metadata_content = example_metadata_content[list(keeper_cols)]

green_content = example_metadata_content[green_id_col + additional_green_col_names]

green_content

,ID,sample_name,investigation_type,package,source_mat_ID
3,UUID:472894-473947-847398,P4_-0-10_4C,genome; metabolome,soil,NaN
4,UUID:684267-410686-971057,L10_-20-30_13C-Gluc,transcriptome; metabolome,soil,UUID:472894-473947-847398
5,UUID:472894-473947-847396,6_J2_75-WHC,organic matter,soil,UUID:472894-473947-847398
6,UUID:516394-970067-847398,T4-35,lipidome; 16S-Amplicon; ITS-Amplicon,soil,NaN
7,UUID:472894-473947-642384,2020-05-21_S19,proteome; genome,soil,UUID:516394-970067-847398


In [15]:
red_content = example_metadata_content[green_id_col + red_col_names]

red_content

,ID,growth_facil,geo_loc_name,lat_lon,collection_date,samp_mat_process,store_cond,samp_store_temp,samp_size,env_broad_scale,env_local_scale,env_medium,gold_ecosystem,microbiome_taxonomy
3,UUID:472894-473947-847398,field,"USA; Prosser, Washington; Washington State Uni...","46.251709, -119.728663",2020-05-21T12:00:00,snap freeze in liquid N,fresh,4 degree Celsius,NaN,arid biome,agricultural field,agricultural soil,5424:Environmental:Terrestrial:Soil:Bulk soil:...,soil metagenome
4,UUID:684267-410686-971057,field_incubation,"USA; Prosser, Washington; Washington State Uni...","46.251709, -119.728663",2020-05-21T13:30:00,"2micron sieve, wet to 25% gravimetric water ca...",frozen,-80 degree Celsius,NaN,arid biome,agricultural field,agricultural soil,5424:Environmental:Terrestrial:Soil:Bulk soil:...,soil metagenome
5,UUID:472894-473947-847396,field,"USA; Prosser, Washington; Washington State Uni...","46.251709, -119.728663",2020-05-21 00:00:00,"aggregate separation at optimal moisture, drie...",other,room temperature,NaN,arid biome,agricultural field,agricultural soil,5424:Environmental:Terrestrial:Soil:Bulk soil:...,soil metagenome
6,UUID:516394-970067-847398,field,"USA; Prosser, Washington; Washington State Uni...","46.251500, -119.728483",2021-01-05T15:05:00,"crushed via morter and pestle, temperature acc...",lyophilized,-80 degree Celsius,NaN,arid biome,agricultural field,agricultural soil,5424:Environmental:Terrestrial:Soil:Bulk soil:...,soil metagenome
7,UUID:472894-473947-642384,field,"USA; Prosser, Washington; Washington State Uni...","46.251500, -119.728483",2021-03-11T09:25:00,ground via omni bead beater,frozen,-20 degree Celsius,NaN,arid biome,agricultural field,agricultural soil,5424:Environmental:Terrestrial:Soil:Bulk soil:...,soil metagenome


In [19]:
blue_cols = list((set(keeper_cols) - set(green_id_col)) - set(red_col_names))
blue_content = example_metadata_content[green_id_col + blue_cols]
blue_content

,ID,treatment.1,investigation_type,source_mat_ID,samp_collect_device,depth,depth.1,sample_name,size_fract,package,treatment.3,treatment.2,treatment,elev
3,UUID:472894-473947-847398,NaN,genome; metabolome,NaN,"11/2' x 6""; stainless stee",0-1,NaN,P4_-0-10_4C,4mm,soil,NaN,NaN,NaN,500.0
4,UUID:684267-410686-971057,1000µg of C (glucose)/g soil,transcriptome; metabolome,UUID:472894-473947-847398,"11/2' x 6""; stainless stee",0-1,NaN,L10_-20-30_13C-Gluc,2mm,soil,NaN,13C Glucose,NaN,500.0
5,UUID:472894-473947-847396,1000µg of C (glucose)/g soil,organic matter,UUID:472894-473947-847398,"11/2' x 6""; stainless stee",0-1,NaN,6_J2_75-WHC,2mm,soil,75% water holding capacity; moisture maintaine...,NaN,NaN,500.0
6,UUID:516394-970067-847398,NaN,lipidome; 16S-Amplicon; ITS-Amplicon,NaN,"11/2' x 6""; stainless stee",NaN,-20-30,T4-35,0.5-.25mm,soil,NaN,NaN,35C,1000.0
7,UUID:472894-473947-642384,NaN,proteome; genome,UUID:516394-970067-847398,"11/2' x 6""; stainless stee",NaN,-20-30,2020-05-21_S19,1-.05mm,soil,NaN,NaN,NaN,1000.0
